# Libraries

In [ ]:
!pip install unidecode

In [ ]:
!sudo add-apt-repository -y ppa:fbirlik/sqlite3
!sudo apt update
!sudo apt-get install -y sqlite3

In [2]:
import os
import sqlite3
import requests
import pandas as pd

from unidecode import unidecode

# Carregando dados VRA

In [3]:
# Get all files
path = '/content/drive/MyDrive/data_eleflow/VRA/'
files = [
  path + file for file in os.listdir(path)
]

# Concat data
data = []
for file in files:
  data.append(pd.read_json(file, encoding='utf-8-sig'))
vra_data = pd.concat(data)

# Rename Header -> snake_case
header_name = {
    "ICAOEmpresaAérea": "icao_empresa_aerea",
    "NúmeroVoo": "numero_voo",
    "CódigoAutorização": "codigo_autorizacao",
    "CódigoTipoLinha": "codigo_tipo_linha",
    "ICAOAeródromoOrigem": "icao_aerodromo_origem",
    "ICAOAeródromoDestino": "icao_aerodromo_destino",
    "PartidaPrevista": "partida_prevista",
    "PartidaReal": "partida_real",
    "ChegadaPrevista": "chegada_prevista",
    "ChegadaReal": "chegada_real",
    "SituaçãoVoo": "situacao_voo",
    "CódigoJustificativa": "codigo_justificativa"
}
vra_data.rename(columns=header_name, inplace=True)
vra_data

,icao_empresa_aerea,numero_voo,codigo_autorizacao,codigo_tipo_linha,icao_aerodromo_origem,icao_aerodromo_destino,partida_prevista,partida_real,chegada_prevista,chegada_real,situacao_voo,codigo_justificativa
0,AEA,0057,0,X,LEMD,SBGR,2021-10-01 18:45:00,2021-10-01 19:24:00,2021-10-02 05:25:00,2021-10-02 05:06:00,REALIZADO,N/A
1,AEA,0057,0,X,LEMD,SBGR,2021-10-03 18:45:00,2021-10-03 20:27:00,2021-10-04 05:25:00,2021-10-04 05:39:00,REALIZADO,N/A
2,AEA,0057,0,X,LEMD,SBGR,2021-10-06 18:45:00,2021-10-06 19:18:00,2021-10-07 05:25:00,2021-10-07 05:05:00,REALIZADO,N/A
3,MWM,5800,0,C,SBBR,SBEG,2021-10-12 01:25:00,2021-10-12 01:22:00,2021-10-12 04:25:00,2021-10-12 04:18:00,REALIZADO,N/A
4,MWM,5800,0,C,SBBR,SBEG,2021-10-13 01:25:00,None,2021-10-13 04:25:00,None,CANCELADO,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
40950,AZU,4411,0,N,SBCF,SBMA,2021-06-28 23:00:00,2021-06-28 22:51:00,2021-06-29 01:35:00,2021-06-29 01:23:00,REALIZADO,N/A
40951,AZU,4411,0,N,SBCF,SBMA,2021-06-29 23:00:00,2021-06-29 22:58:00,2021-06-30 01:35:00,2021-06-30 01:32:00,REALIZADO,N/A
40952,AZU,4411,0,N,SBCF,SBMA,2021-06-30 23:00:00,2021-06-30 22:56:00,2021-07-01 01:35:00,2021-07-01 01:33:00,REALIZADO,N/A
40953,AZU,4412,0,N,SBCF,SBVT,2021-06-01 21:40:00,2021-06-01 21:36:00,2021-06-01 22:45:00,2021-06-01 22:30:00,REALIZADO,N/A


# Get all ICAO aerodromos

In [4]:
icao_codes = set()

for origin_icao, destiny_icao in zip(vra_data['icao_aerodromo_origem'], vra_data['icao_aerodromo_destino']):
  icao_codes.add(origin_icao)
  icao_codes.add(destiny_icao)

icao_codes

{'CYHM',
 'CYUL',
 'CYYZ',
 'DGAA',
 'DIAP',
 'DNKN',
 'DNMM',
 'EBBR',
 'EBLG',
 'EBOS',
 'EDDF',
 'EDDK',
 'EDFH',
 'EGBB',
 'EGLL',
 'EGNX',
 'EGSS',
 'EHAM',
 'EHBK',
 'EKCH',
 'ELLX',
 'FAOR',
 'FNLU',
 'FQMA',
 'FYWH',
 'FZAA',
 'GBYD',
 'GCTS',
 'GMAD',
 'GMMN',
 'GOBD',
 'GUCY',
 'GVAC',
 'HAAB',
 'HKJK',
 'HLLM',
 'HSSS',
 'KAEX',
 'KATL',
 'KAUH',
 'KAUS',
 'KBOS',
 'KCVG',
 'KDFW',
 'KEWR',
 'KFLL',
 'KHSV',
 'KIAD',
 'KIAH',
 'KIWA',
 'KJFK',
 'KLAX',
 'KMEM',
 'KMIA',
 'KORD',
 'KPUC',
 'KPVG',
 'KSTL',
 'LEBL',
 'LEMD',
 'LEZG',
 'LFLX',
 'LFOK',
 'LFPG',
 'LIMC',
 'LIRF',
 'LKMT',
 'LPPR',
 'LPPT',
 'LRCK',
 'LSZH',
 'LTBA',
 'LTFM',
 'LYBE',
 'MDPC',
 'MDSD',
 'MGGT',
 'MKJP',
 'MMGL',
 'MMMD',
 'MMMX',
 'MMMY',
 'MMUN',
 'MNMG',
 'MPTO',
 'MSLP',
 'MTPP',
 'MUHA',
 'NZAA',
 'OEJN',
 'OMAA',
 'OMDB',
 'OMSJ',
 'OOMS',
 'OTHH',
 'PANC',
 'RJAA',
 'RKSI',
 'SAAR',
 'SABE',
 'SACO',
 'SAEZ',
 'SAME',
 'SANT',
 'SANU',
 'SARI',
 'SBAC',
 'SBAE',
 'SBAR',
 'SBAT',
 'SBAU',
 

# Carregando dados AIR_CIA

In [61]:
# Get all files
path = '/content/drive/MyDrive/data_eleflow/AIR_CIA/'
files = [
  path + file for file in os.listdir(path)
]

# Concat data
data = []
for file in files:
  data.append(pd.read_csv(file, delimiter=';'))
air_cia_data = pd.concat(data)

# Rename Header -> snake_case
header_name = {}
for header in air_cia_data.columns:
  header_name[header] = unidecode(header).lower().replace(' ', '_')
air_cia_data.rename(columns=header_name, inplace=True)

# Split ICAO IATA
air_cia_data[['icao', 'iata']] = air_cia_data['icao_iata'].str.split(' ', expand=True)
del air_cia_data['icao_iata']


air_cia_data = air_cia_data.drop_duplicates()
air_cia_data

,razao_social,cnpj,atividades_aereas,endereco_sede,telefone,e-mail,decisao_operacional,data_decisao_operacional,validade_operacional,icao,iata
0,RIMA - RIO MADEIRA AEROTÁXI LTDA.,04.778.630/0001-42,"LIGAÇÃO AÉREA SISTEMÁTICA REGULAR, TÁXI-AÉREO","AV. LAURO SODRÉ, Nº 6490 - AEROPORTO INTERNACI...",(69) 3225-8000,NaN,DECISÃO Nº 116,26/07/2017,27/07/2022,RIM,0R
1,RTS - ROTA DO SOL TÁXI AÉREO LTDA.,01.904.715/0001-31,"LIGAÇÃO AÉREA SISTEMÁTICA REGULAR, SERVIÇOS AÉ...","PRAÇA EDUARDO GOMES, S/Nº, TERMINAL DE AVIAÇÃO...",(85) 3272-3555,quadros@voerotadosol.com,DECISÃO Nº 21,13/02/2019,15/02/2024,CKP,None
0,ABSA - AEROLINHAS BRASILEIRAS S.A.,00.074.635/0001-33,TRANSPORTE AÉREO REGULAR,"AEROPORTO INTERNACIONAL DE VIRACOPOS, RODOVIA ...",(11) 5582-8055,gar@tam.com.br,DECISÃO Nº 41,22/04/2015,23/04/2025,LTG,M3
1,AEROSUL TÁXI AÉREO LTDA (EX.: AUSTEN TÁXI AÉRE...,27.315.694/0001-02,"SERVIÇOS AÉREOS PÚBLICOS, TÁXI-AÉREO, TRANSPOR...","RODOVIA PR 218, KM 7, JARDIM UNIVERSITÁRIO, AE...",(43) 3176-4030,operacoes@aerosul.com.br,DECISÃO Nº 282,10/02/2021,NaN,ASO,2S
2,ASTA LINHAS AÉREAS LTDA ( EX - AMÉRICA DO SUL ...,12.703.737/0001-50,"SERVIÇOS AÉREOS PÚBLICOS, TRANSPORTE AÉREO REG...","AV GOVERNADOR JOÃO PONCE DE ARRUDA, S/N, HANGA...",(65) 3614-2684 / (18) 2104-1000,NaN,DECISÃO Nº 115,31/07/2019,02/08/2024,SUL,0A
3,ATA - AEROTÁXI ABAETÉ LTDA.,14.674.451/0001-19,"TÁXI-AÉREO, TRANSPORTE AÉREO NÃO REGULAR, TRAN...","PÇA. GAGO COUTINHO, S/N, AEROPORTO INTERNACIO...",(71) 34629600 / (71) 996643563,guilherme@abaete.com.br,DECISÃO Nº 36,05/03/2020,NaN,ABJ,None
4,AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA),04.263.318/0001-16,TRANSPORTE AÉREO REGULAR,"AVENIDA EMILIO ANTONON, 901, BAIRRO CHÁCARA AE...",(11) 45822355/ (11) 987433988,NaN,"Decisão nº 14, de 25/01/2018",25/01/2018,29/01/2028,ACN,2F
5,AZUL LINHAS AÉREAS BRASILEIRAS S/A,09.296.295/0001-60,TRANSPORTE AÉREO REGULAR,"AV. MARCOS PENTEADO ULHÔA RODRIGUES,N°939, 9° ...",(55) (11) 4134-9887 | Fax: (55)(11)4134-9890,vanessa.reis@voeazul.com.br,Decisão nº 178,20/11/2018,21/11/2028,AZU,AD
6,GOL LINHAS AÉREAS S.A. (EX- VRG LINHAS AÉREAS ...,07.575.651/0001-59,TRANSPORTE AÉREO REGULAR,"PRAÇA SENADOR SALGADO FILHO, S/N°, AEROPORTO S...",(11) 5098-2990 / (11) 5098-7872,rddfonseca@voegol.com.br,DECISÃO Nº 156,05/11/2019,NaN,GLO,G3
7,MAP TRANSPORTES AÉREOS LTDA.,10.483.635/0001-40,TRANSPORTE AÉREO REGULAR,"AVENIDA SANTOS DUMONT, Nº 1350, AEROPORTO INTE...",(92) 36521620,contabil@voemap.com.br,DECISÃO Nº 89,14/08/2012,17/08/2022,PAM,7M


# GET API Airport info

In [62]:
with requests.session() as session:
  url = 'https://airport-info.p.rapidapi.com/airport'
  url_head = {
    'X-RapidAPI-Key': '4c13e6c3eemsh73f49fcc12e4e57p1483c0jsn7d15bfd7c8ff',
    'X-RapidAPI-Host': 'airport-info.p.rapidapi.com',
  }
  session.headers.update(url_head)

  airport_infos = list()
  for icao in icao_codes:
    url_par = {'icao': icao}
    response = session.get(
        url, headers=url_head, params=url_par
    )

    if response.status_code != 200:
        msg = f'Airtable API returned {response.status_code}'
        print(msg)

    airport_infos.append(response.json())

airport_infos = pd.DataFrame(airport_infos)
airport_infos

,id,iata,icao,name,location,street_number,street,city,county,state,country_iso,country,postal_code,phone,latitude,longitude,uct,website,error
0,268.0,AMS,EHAM,Amsterdam Airport Schiphol,"Amsterdam, Netherlands",202,Evert van de Beekstraat,Schiphol,Haarlemmermeer,Noord-Holland,NL,Netherlands,1118 CP,+31 20 794 0800,52.310540,4.768274,120.0,http://www.schiphol.nl/,NaN
1,1399.0,CMB,VCBI,Bandaranaike International Airport,"Colombo, Sri Lanka",,Canada Friendship Rd,කටුනායක,ගම්පහ,"බස්නාහිර පළාත, ශ් රී ලංකාව",LK,Sri Lanka,11450,+94 112 264 444,7.180155,79.884254,330.0,http://www.airport.lk/,NaN
2,3290.0,ISL,LTFM,Istanbul New Airport,"Istanbul, Turkey",,,,Bakırköy,Istanbul,TR,Turkey,34149,+90 212 463 30 00,40.982990,28.810442,180.0,http://www.ataturkairport.com/,NaN
3,212.0,AKL,NZAA,Auckland Airport,"Auckland, New Zealand",,Ray Emery Drive,Auckland,,Auckland,NZ,New Zealand,2022,+64 9-275 0789,-37.008247,174.785030,780.0,http://www.aucklandairport.co.nz/,NaN
4,1141.0,CAF,SWCA,Carauari Airport,"Carauari, Amazonas, Brazil",600-628,Avenida Dom Pedro Segundo,,Carauari,Amazonas,BR,Brazil,69500-000,,-4.875501,-66.899160,-240.0,,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,1367.0,CKS,SBCJ,Carajás Airport,"Parauapebas, Pará, Brazil",,,,Parauapebas,Pará,BR,Brazil,68515-000,+55 94 98403-1335,-6.114454,-50.001007,-180.0,http://www.infraero.gov.br/,NaN
311,3395.0,JDO,SBJU,Juazeiro do Norte Airport,"Juazeiro do Norte, Ceará, Brazil",4000,Avenida Virgílio Távora,,Juazeiro do Norte,Ceará,BR,Brazil,63020-735,+55 88 3311-6500,-7.215097,-39.272230,-180.0,http://www4.infraero.gov.br/aeroportos/aeropor...,NaN
312,4202.0,LIS,LPPT,Lisbon Portela Airport,"Lisbon, Portugal",,Alameda das Comunidades Portuguesas,Lisboa,,Lisboa,PT,Portugal,1700-111,+351 21 841 3500,38.775593,-9.135366,60.0,http://www.ana.pt/pt-PT/Aeroportos/Lisboa/Lisb...,NaN
313,4838.0,MOC,SBMK,Montes Claros/Mário Ribeiro Airport,"Montes Claros, Minas Gerais, Brazil",,Avenida Comandante João Milton,,Montes Claros,Minas Gerais,BR,Brazil,39404-844,+55 38 3229-1200,-16.704617,-43.820087,-180.0,http://www4.infraero.gov.br/aeroportos/aeropor...,NaN


# Creating Data Base

In [ ]:
conn = sqlite3.connect('eleflow.db')
cursor = conn.cursor()

create_airport_infos = """
CREATE TABLE airport_infos (
  id INTEGER PRIMARY KEY,
  iata VARCHAR(20),
  icao VARCHAR(20),
  name VARCHAR(100),
  location VARCHAR(100),
  street_number VARCHAR(20),
  street VARCHAR(50),
  city VARCHAR(50),
  county VARCHAR(50),
  state VARCHAR(20),
  country_iso VARCHAR(2),
  country VARCHAR(100),
  postal_code VARCHAR(100),
  phone VARCHAR(100),
  latitude NUMERIC,
  longitude NUMERIC,
  uct NUMERIC,
  website VARCHAR(100),
  error VARCHAR(100)
);
"""

cursor.execute(create_airport_infos)

create_vra = """
CREATE TABLE vra (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  icao_empresa_aerea VARCHAR(20),
  numero_voo VARCHAR(20),
  codigo_autorizacao VARCHAR(20),
  codigo_tipo_linha VARCHAR(2),
  icao_aerodromo_origem VARCHAR(20),
  icao_aerodromo_destino VARCHAR(20),
  partida_prevista TIMESTAMP,
  partida_real TIMESTAMP,
  chegada_prevista TIMESTAMP,
  chegada_real TIMESTAMP,
  situacao_voo VARCHAR(20),
  codigo_justificativa VARCHAR(20)
);
"""

cursor.execute(create_vra)

create_air_cia = """
CREATE TABLE air_cia (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  razao_social VARCHAR(50),
  cnpj VARCHAR(30),
  atividades_aereas VARCHAR(50),
  endereco_sede VARCHAR(50),
  telefone VARCHAR(20),
  email VARCHAR(30),
  decisao_operacional VARCHAR(50),
  data_decisao_operacional date,
  validade_operacional date,
  icao VARCHAR(10),
  iata VARCHAR(10)
);
"""

cursor.execute(create_air_cia)

# Insert data in DB

### Airport Info

In [54]:
values = list()
for index, row in airport_infos.iterrows():
  if str(row['icao']) == 'nan':
    continue
 
  values.append("({}, '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', {}, {}, {}, '{}', '{}')".format(
    int(row['id']),
    row['iata'],
    row['icao'],
    row['name'].replace("'", "''"),
    row['location'].replace("'", "''"),
    row['street_number'],
    row['street'].replace("'", "''"),
    row['city'].replace("'", "''"),
    row['county'].replace("'", "''"),
    row['state'].replace("'", "''"),
    row['country_iso'],
    row['country'].replace("'", "''"),
    row['postal_code'],
    row['phone'],
    row['latitude'],
    row['longitude'],
    row['uct'],
    row['website'],
    row['error']
  ))

sql = 'insert into airport_infos(id,iata,icao,name,location,street_number,street,city,county,state,country_iso,country,postal_code,phone,latitude,longitude,uct,website,error) values\n'
sql += ','.join(values) + ';'
cursor.execute(sql)

### VRA

In [12]:
values = list()
for index, row in vra_data.iterrows():

  values.append("('{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}')".format(
    row['icao_empresa_aerea'],
    row['numero_voo'],
    row['codigo_autorizacao'],
    row['codigo_tipo_linha'],
    row['icao_aerodromo_origem'],
    row['icao_aerodromo_destino'],
    row['partida_prevista'],
    row['partida_real'],
    row['chegada_prevista'],
    row['chegada_real'],
    row['situacao_voo'],
    row['codigo_justificativa'],
  ))

sql_vra = 'insert into vra(icao_empresa_aerea,numero_voo,codigo_autorizacao,codigo_tipo_linha,icao_aerodromo_origem,icao_aerodromo_destino,partida_prevista,partida_real,chegada_prevista,chegada_real,situacao_voo,codigo_justificativa) values\n'
sql_vra += ','.join(values) + ';'
cursor.execute(sql_vra)

### AIR CIA

In [66]:
values = list()
for index, row in air_cia_data.iterrows():
 
  values.append("('{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}')".format(
    row['razao_social'],
    row['cnpj'],
    row['atividades_aereas'],
    row['endereco_sede'],
    row['telefone'],
    row['e-mail'],
    row['decisao_operacional'],
    row['data_decisao_operacional'],
    row['validade_operacional'],
    row['icao'],
    row['iata']
  ))

sql = 'insert into air_cia(razao_social,cnpj,atividades_aereas,endereco_sede,telefone,email,decisao_operacional,data_decisao_operacional,validade_operacional,icao,iata) values\n'
sql += ','.join(values) + ';'
cursor.execute(sql)

# Checking DB

In [74]:
cursor.execute('select count(*) from airport_infos;')
airport_db = cursor.fetchall()

cursor.execute("select count(*) from vra;")
vra_db = cursor.fetchall()

cursor.execute("select count(*) from air_cia;")
air_cia_db = cursor.fetchall()

In [75]:
airport_db

[(276,)]

In [76]:
vra_db

[(535803,)]

In [77]:
air_cia_db

[(17,)]

# SQL VIEWS

In [69]:
first_query = """
WITH base as (
  SELECT
    icao_empresa_aerea,
    icao_aerodromo_origem,
    icao_aerodromo_destino,
    ROW_NUMBER() OVER(
      PARTITION BY icao_empresa_aerea
      ORDER BY count(*) DESC
    ) as row_
  FROM vra
  GROUP BY 1,2,3
  order by 1 DESC
)
SELECT
  COALESCE(air_cia.razao_social, base.icao_empresa_aerea) as 'Companhia Aérea',
  COALESCE(ai1.name, base.icao_aerodromo_origem) as 'Aeroporto Origem',
  base.icao_aerodromo_origem as 'ICAO Aeroporto Origem',
  ai1.state as 'Estado Aeroporto Origem',
  COALESCE(ai2.name, base.icao_aerodromo_destino) as 'Aeroporto Destino',
  base.icao_aerodromo_destino as 'ICAO Aeroporto Destino',
  ai2.state as 'Estado Aeroporto Destino'
FROM base
LEFT JOIN air_cia
  on base.icao_empresa_aerea = air_cia.icao
LEFT JOIN airport_infos ai1
  on base.icao_aerodromo_origem = ai1.icao
LEFT JOIN airport_infos ai2
  on base.icao_aerodromo_destino = ai2.icao
WHERE base.row_ = 1
"""

first_result = pd.read_sql_query(first_query, conn)
first_result

,Companhia Aérea,Aeroporto Origem,ICAO Aeroporto Origem,Estado Aeroporto Origem,Aeroporto Destino,ICAO Aeroporto Destino,Estado Aeroporto Destino
0,VVC,El Dorado International Airport,SKBO,Bogotá,Tancredo Neves International Airport (Confins ...,SBCF,Minas Gerais
1,VDA,Recife/Guararapes–Gilberto Freyre Internationa...,SBRF,Pernambuco,Rio de Janeiro–Galeão International Airport,SBGL,Rio de Janeiro
2,UPS,Miami International Airport,KMIA,Florida,Viracopos/Campinas International Airport,SBKP,São Paulo
3,UKL,George Bush Intercontinental Airport,KIAH,Texas,Norman Manley International Airport,MKJP,Kingston Parish
4,UAL,São Paulo–Guarulhos International Airport,SBGR,São Paulo,Newark Liberty International Airport,KEWR,New Jersey
...,...,...,...,...,...,...,...
105,AEA,Adolfo Suárez Madrid–Barajas Airport,LEMD,Comunidad de Madrid,São Paulo–Guarulhos International Airport,SBGR,São Paulo
106,ADB,Johan Adolf Pengel International Airport,SMJP,Para,Val de Cans International Airport,SBBE,Pará
107,AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA),Coari Airport,SWKO,State of Amazonas,Eduardo Gomes International Airport,SBEG,Amazonas
108,ACA,São Paulo–Guarulhos International Airport,SBGR,São Paulo,Toronto Pearson International Airport,CYYZ,Ontario


In [71]:
second_query = """
WITH RECURSIVE split(icao_empresa_aerea, aeroporto, str) AS (
    SELECT icao_empresa_aerea, '', aeroportos||';'
    FROM (
      SELECT
        icao_empresa_aerea,
        icao_aerodromo_origem || ';' || icao_aerodromo_destino as aeroportos
      FROM vra
    )
    UNION ALL
    SELECT
      icao_empresa_aerea,
      substr(str, 0, instr(str, ';')),
      substr(str, instr(str, ';') +1)
    FROM split WHERE str!=''
), base as (
  SELECT
    icao_empresa_aerea,
    aeroporto
  FROM split
  WHERE aeroporto != ''
), max as (
  SELECT
    aeroporto,
    icao_empresa_aerea,
    total_dp
  FROM (
      SELECT
        aeroporto,
        icao_empresa_aerea,
        count(*) as total_dp,
        ROW_NUMBER() OVER(
            PARTITION BY aeroporto
            ORDER BY count(*) DESC
          ) as row_
      FROM base
      GROUP BY 1, 2
  )
  WHERE row_ = 1
), partidas as (
  SELECT
    max.aeroporto,
    max.icao_empresa_aerea,
    count(*) as partidas
  FROM max
  JOIN vra
    on max.aeroporto = vra.icao_aerodromo_origem
    and max.icao_empresa_aerea = vra.icao_empresa_aerea
  GROUP BY 1, 2
), chegadas as (
  SELECT
    max.aeroporto,
    max.icao_empresa_aerea,
    count(*) as chegadas
  FROM max
  JOIN vra
    on max.aeroporto = vra.icao_aerodromo_destino
    and max.icao_empresa_aerea = vra.icao_empresa_aerea
  GROUP BY 1, 2
)
SELECT
  COALESCE(airport_infos.name, max.aeroporto) as 'Aeroporto',
  max.aeroporto as 'ICAO Aeroporto',
  COALESCE(air_cia.razao_social, max.icao_empresa_aerea) as 'Companhia Aérea',
  max.total_dp as 'Total de Decolagens e Pousos',
  CAST(partidas.partidas as INTERGER) as 'Total de Decolagens',
  CAST(chegadas.chegadas as INTERGER) as 'Total de Pousos'
FROM max
LEFT JOIN airport_infos
  on max.aeroporto = airport_infos.icao
LEFT JOIN air_cia
  on max.icao_empresa_aerea = air_cia.icao
LEFT JOIN partidas
  on max.aeroporto = partidas.aeroporto
LEFT JOIN chegadas
  on max.aeroporto = chegadas.aeroporto
ORDER BY 1;
"""

second_result = pd.read_sql_query(second_query, conn)
second_result

,Aeroporto,ICAO Aeroporto,Companhia Aérea,Total de Decolagens e Pousos,Total de Decolagens,Total de Pousos
0,Abu Dhabi International Airport,OMAA,ETD,12,6.0,6.0
1,Adolfo Suárez Madrid–Barajas Airport,LEMD,IBE,656,331.0,325.0
2,Afonso Pena International Airport,SBCT,AZUL LINHAS AÉREAS BRASILEIRAS S/A,12352,6177.0,6175.0
3,Agadir–Al Massira Airport,GMAD,MEM,2,1.0,1.0
4,Alberto Alcolumbre International Airport,SBMQ,GOL LINHAS AÉREAS S.A. (EX- VRG LINHAS AÉREAS ...,1327,663.0,664.0
...,...,...,...,...,...,...
310,Washington Dulles International Airport,KIAD,UAL,8,4.0,4.0
311,Wuhan Tianhe International Airport,ZHHH,ETH,4,4.0,NaN
312,Zaragoza Airport,LEZG,QTR,6,3.0,3.0
313,Zumbi dos Palmares International Airport,SBMO,AZUL LINHAS AÉREAS BRASILEIRAS S/A,4372,2186.0,2186.0
